# 分词之最大双向匹配算法

最大双向匹配算法是利用词典进行分词的一个算法，融合了前向前向最大匹配和后向最大匹配。

前向最大匹配就是从前往后，每次都取出词典中存在的最长的词，后向匹配则与之相反，最后在结合时，根据分词结果中单个字的数量、非登录词的数量和词的总数量，从两个结果中选出一个。代码如下：

In [1]:
# 词典中词的最大长度
window_size = 4


def fmm(source, words_dict):
    len_source = len(source)
    index = 0
    words = []  # 存储分词结果
 
    while index < len_source:
        match = False
        for i in range(window_size, 0, -1):
            sub_str = source[index: index+i]
            if sub_str in words_dict:
                match = True
                words.append(sub_str)
                index += i
                break
        if not match:
            words.append(source[index])
            index += 1
    return words


def bmm(source, words_dict):
    len_source = len(source)  # 原句长度
    index = len_source
    words = []  # 分词后句子每个词的列表
 
    while index > 0:
        match = False
        for i in range(window_size, 0, -1):
            sub_str = source[index-i: index]
            if sub_str in words_dict:
                match = True
                words.append(sub_str)
                index -= i
                break
        if not match:
            words.append(source[index-1])
            index -= 1
    words.reverse()  # 得到的列表倒序
    return words
    

def bi_mm(source, words_dict):
    forward = fmm(source, words_dict)
    backward = bmm(source, words_dict)
    # 正反向分词结果
    print("FMM: ", forward)
    print("BMM: ", backward)
    # 单字词个数
    f_single_word = 0
    b_single_word = 0
    # 总词数
    tot_fmm = len(forward)
    tot_bmm = len(backward)
    # 非字典词数
    oov_fmm = 0
    oov_bmm = 0
    # 罚分，罚分值越低越好
    score_fmm = 0
    score_bmm = 0
    # 如果正向和反向结果一样，返回任意一个
    if forward == backward:
        return backward
    # print(backward)
    else:  # 分词结果不同，返回单字数、非字典词、总词数少的那一个
        for each in forward:
            if len(each) == 1:
                f_single_word += 1
        for each in backward:
            if len(each) == 1:
                b_single_word += 1
        for each in forward:
            if each not in words_dict:
                oov_fmm += 1
        for each in backward:
            if each not in backward:
                oov_bmm += 1
        # 可以根据实际情况调整惩罚分值
        # 这里都罚分都为1分
        # 非字典词越少越好
        if oov_fmm > oov_bmm:
            score_bmm += 1
        elif oov_fmm < oov_bmm:
            score_fmm += 1
        # 总词数越少越好
        if tot_fmm > tot_bmm:
            score_bmm += 1
        elif tot_fmm < tot_bmm:
            score_fmm += 1
        # 单字词越少越好
        if f_single_word > b_single_word:
            score_bmm += 1
        elif f_single_word < b_single_word:
            score_fmm += 1
 
        # 返回罚分少的那个
        if score_fmm < score_bmm:
            return forward
        else:
            return backward

In [2]:
words_dict = ['研究', '研究生', '生命', '起源', '用户', '使用', '满意']
bi_mm("研究生命的起源", words_dict)

FMM:  ['研究生', '命', '的', '起源']
BMM:  ['研究', '生命', '的', '起源']


['研究生', '命', '的', '起源']

In [3]:
bi_mm("使用户满意", words_dict)

FMM:  ['使用', '户', '满意']
BMM:  ['使', '用户', '满意']


['使用', '户', '满意']